# **TABELA FILIAL**

## <u>Realizando as transformações necessárias e criando nova tabela para ser consumida</u>

- Criaremos a coluna `Ordem` para ordernar as filiais uma vez que a PK da tabela staging é do tipo texto. 
- Além disso, preencheremos as informações nulas com informação da JC Brothers

In [3]:
CREATE OR ALTER PROCEDURE SP_CARREGA_FILIAL
	@CODFILIAL NCHAR(2),
	@EMPRESA NVARCHAR(150),
	@FILIAL NVARCHAR(25)
AS
BEGIN    
----Faz a transformação das informações
	DECLARE @ORDEM SMALLINT = CONVERT(SMALLINT, @CODFILIAL)
	DECLARE @DTUPDATE DATETIME = DATEADD(hour, -3, GETUTCDATE())

	SET @EMPRESA  = CASE WHEN @EMPRESA = 'NULL' THEN 'JC BROTHERS' ELSE @EMPRESA END
	SET @FILIAL  =  CASE WHEN @FILIAL = 'NULL' THEN  'JC BROTHERS' ELSE @FILIAL END
	
----faz o merge das informações
	IF EXISTS (SELECT 1 FROM FILIAL WHERE [CodFilial] = @CODFILIAL)
		BEGIN
		UPDATE FILIAL SET EMPRESA = @EMPRESA, FILIAL = @FILIAL, DTUPDATE = @DTUPDATE WHERE [CodFilial] = @CODFILIAL
	END
	ELSE
	BEGIN
		INSERT INTO FILIAL ([CodFilial], Empresa, Filial, Ordem, DtUpdate)
		VALUES (@CODFILIAL, @EMPRESA, @FILIAL, @ORDEM, @DTUPDATE)
	END
END

Commands completed successfully.

Total execution time: 00:00:00.039

In [5]:
SELECT * FROM FILIAL

(0 rows affected)

Total execution time: 00:00:00.016

CodFilial,Empresa,Filial,Ordem,DtUpdate


In [7]:
CREATE OR ALTER PROCEDURE SP_RETORNA_FILIAL
    @CODFILIAL NCHAR(2)
AS
BEGIN    

    IF EXISTS (SELECT 1 FROM FILIAL WHERE CODFILIAL = @CODFILIAL)
	SELECT FORMAT(DTUPDATE, 'yyyy-MM-dd HH:mm:ss') DTUPDATE FROM FILIAL WHERE CODFILIAL = @CODFILIAL
END

Commands completed successfully.

Total execution time: 00:00:00.028